# Bathymetry Subtidal
Tests subtidal bathymetry functions of `bathymetry.py`.

In [2]:
from pathlib import Path
import sys
import time

import ee
import geemap
from geojson import Polygon

sys.path.append(str(Path.cwd().parent.parent))

from eepackages.applications import bathymetry

In [3]:
Map = geemap.Map(center=(52.97, 4.74), zoom=11)
bounds = ee.Geometry(Polygon([[
    (4.574518432617185, 52.91662291147939),
    (4.90548156738281, 52.91662291147939),
    (4.90548156738281, 53.02331125248889),
    (4.574518432617185, 53.02331125248889),
    (4.574518432617185, 52.91662291147939)
]]))
scale = Map.getScale()
# scale = 30
start_date = "2019-01-01"
stop_date = "2021-01-01"

sdb = bathymetry.Bathymetry()
Map.addLayer(bounds)

Enter verification code: 4/1AX4XfWiYjfIJHyuIwKPaXUlJiAVni2djX6SanN70SMIkTQUID1A0shzb8ag

Successfully saved authorization token.


In [4]:
# Run interactively
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
    skip_neighborhood_search=False
)
Map.addLayer(image)# .reproject(ee.Projection('EPSG:3857').atScale(90)))  # without projection, takes default projection
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

Cleanup tests assets for this notebook

In [ ]:
def cartesian_product(l1: ee.List, l2: ee.List) -> ee.List:
    return l1.map(lambda e1: l2.map(lambda e2: ee.List([e1, e2])).flatten())

In [ ]:
user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
asset_id = f"users/{user_name}/tests/test_inverse_depth"
asset_dict = ee.data.listAssets({"parent": f"projects/earthengine-legacy/assets/users/{user_name}/tests"})
if "tests" in asset_dict["assets"]:
    ee.data.deleteAsset(asset_id)
ee.data.create_assets(asset_ids=["/".join(asset_id.split("/")[:-1])], asset_type="Folder", mk_parents=True)

In [ ]:
# Let's use the ee.batch.Export class' static methods to create assets
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
    skip_neighborhood_search=False # adds a lot of computation time if False
) #  .reproject(ee.Projection('EPSG:3857').atScale(90))

task = ee.batch.Export.image.toAsset(
    image,
    description="test_compute_inverse_depth",
    region=bounds,
    assetId=asset_id,
    scale=scale,
    crs="EPSG:3857",
    maxPixels=1e11
)

In [ ]:
task.start()

In [ ]:
task_status = task.status()
while task_status['state'] in ['READY', 'RUNNING']:
    print(task_status)
    time.sleep(30)
    task_status = task.status()
else:
    print(task.status())

In [ ]:
image_subtidal = ee.Image(asset_id)
Map.addLayer(image_subtidal, {"gamma": 1.2}, 'RGB_raw', True)
Map